In [3]:
!pip install --upgrade pymorphy2-dicts
!pip install --upgrade DAWG-Python
!pip install --upgrade pymorphy2[fast]
!pip install --upgrade nltk

Requirement already up-to-date: pymorphy2-dicts in c:\users\acer\anaconda3\lib\site-packages (2.4.393442.3710985)
Requirement already up-to-date: DAWG-Python in c:\users\acer\anaconda3\lib\site-packages (0.7.2)
Requirement already up-to-date: pymorphy2[fast] in c:\users\acer\anaconda3\lib\site-packages (0.8)
  Created wheel for nltk: filename=nltk-3.5-py3-none-any.whl size=1434680 sha256=3d4e7aa45cc5e6e32f241941e0e4e9bb1bc9de193f8e3ac8ebd1ffa274ec1c91
  Stored in directory: c:\users\acer\appdata\local\pip\cache\wheels\45\6c\46\a1865e7ba706b3817f5d1b2ff7ce8996aabdd0d03d47ba0266
Successfully built nltk
  Attempting uninstall: nltk
    Found existing installation: nltk 3.4.4
    Uninstalling nltk-3.4.4:
      Successfully uninstalled nltk-3.4.4


In [12]:
from collections import defaultdict,namedtuple
from functools import reduce
from zipfile import ZipFile
from math import log
import heapq
import numpy as np

import pickle
from time import time

import nltk
nltk.download('stopwords')

import pymorphy2
morph = pymorphy2.MorphAnalyzer()

from nltk.corpus import stopwords as stpw

великий


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Acer\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
print(morph.parse('величайший')[0].normal_form)

великий


In [14]:
cash=dict()
stopwords=set(stpw.words())

In [2]:
mysyn = {'сущ' : 'существительное', 'прил' : 'прилагательное', 
         'майнкрафт' : 'minecraft', 'теле2' : 'tele2', 
         'кунг' : 'kung', 'фу' : 'fu', 
         'панда' : 'panda', 'sberbank' : 'сбербанк', 
         'г' : 'год', 'гта' : 'gta', 
         'майн' : 'minecraft', 'сан' : 'san', 
         'андреас' : 'andreas', 'megafon' : 'мегафон', 
         'андроид' : 'android', 'гиг' : 'gb', 
         'виндовс' : 'windows', 'винд' : 'windows', 
         'скайп' : 'skype', 'stalker' : 'сталкер', 
         'ммр' : 'mmr', 'поу' : 'pou', 
         'тандер' : 'thunder', 'вар' : 'war', 
         'бмп' : 'bmp', 'вконтакте' : 'вк', 
         'vk' : 'вк', 'инстаграм' : 'instagram', 
         'пдф' : 'pdf', 'ютуба' : 'youtube', 
         'биос' : 'bios', 'ps' : 'photoshop', 
         'юсб' : 'usb', 'самсунг' : 'samsung', 
         'тег' : 'tag', 'скайрим' : 'skyrim', 
         'дть' : 'demo', 'дискорд' : 'discord', 
         'киви' : 'kiwi', 'симс' : 'sims', 
         'айфон' : 'iphone', 'майкрософт' : 'microsoft', 
         'эксель' : 'excel'}

cash.update(mysyn)

In [ ]:
get_id=dict()
get_url=dict()
with open('urls.numerate.txt') as f:
    for line in f:
        ind,url=line.split('\t')
        url=url.strip()
        
        ind=int(ind)
        get_id[url]=ind
        get_url[ind]=url

In [ ]:
def process_string(s):
    return ''.join(map(lambda x:x.lower() if x.isalpha() or x.isdigit()
                                else ' ' if x not in set(['.','!','?','\n'])
                                else '.',s)) 

def add_cash(word):
    if word not in cash:
        cash[word]=morph.parse(word)[0].normal_form
    if cash[word] in cash:
        return cash[cash[word]]
    return cash[word]
        
def process_string_lem(s):
    return ' '.join(map(add_cash,s.split()))
      
def string_lem(s):
    return process_string_lem(process_string(s)) 

def generate_candidates():
    with open('sample.technosphere.ir1.textrelevance.submission.txt','r') as f:
        f.readline()
        candidates=defaultdict(list)
        reverse_candidates=defaultdict(list)
        for line in f:
            query_id,doc_id=list(map(int,line.strip().split(',')))
            candidates[query_id].append(doc_id)
            reverse_candidates[doc_id].append(query_id)
        return candidates,reverse_candidates

candidates,reverse_candidates=generate_candidates()

In [ ]:
get_query=dict()
get_query_id=dict()
with open('queries.fixed.txt','r',encoding='utf-8-sig') as f:
    for line in f:
        query_id,words=line.split('\t')
        get_query[int(query_id)]=process_string(words).strip('\n').replace('.',' ')
        get_query_id[get_query[int(query_id)]]=int(query_id)